In [1]:
import numpy as np
import pandas as pd
import gdal, osr
import glob
import os
import datetime
from scipy.interpolate import griddata
import re
import xarray as xr
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

/Users/aniket/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/Users/aniket/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
# define variables
TA_da = None
EVI_da = None
LST_Day_da = None
LST_Night_da = None
MIN_PATCH_SIZE = 3
MAX_PATCH_SIZE = 750

In [3]:
# Collect Files
FILE_DICT = {
    'EVI' : glob.glob('MODIS_NDVI/MOD13*EVI*.tif'),
    'EVI_pixel' : glob.glob('MODIS_NDVI/*pixel_reliability*.tif'),
    'EVI_QC' : glob.glob('MODIS_NDVI/*VI_Quality*.tif'),
    'TA' : glob.glob('MODIS_TA/MOD14*FireMask*.tif'),
    'TA_QC' : glob.glob('MODIS_TA/*QA*.tif'),
    'LST_Day' : glob.glob('MODIS_LST_Day/*LST_Day*.tif'),
    'LST_Day_QC' : glob.glob('MODIS_LST_Day/*QC_Day*.tif'),
    'LST_Night' : glob.glob('MODIS_LST_Night/*LST_Night*.tif'),
    'LST_Night_QC' : glob.glob('MODIS_LST_Night/*QC_Night*.tif')
}

In [4]:
# Open files as xarray DataArrays
def open_xarray(data, is_qc=False):
    """
    Opens dataset as an xarray DataArray.
    
    :param data: key from file_dict
    :param file_dict: dictionary of file paths
    
    :returns: xarray data structure"""
    v = FILE_DICT[data]
    pat = r'doy(\d{7})'
    da_list = []
    for fp in v:
        # extract timestamp from filepath
        time = re.findall(pat, fp)[0]
        year = int(time[:4])
        day = int(time[-3:])
        
        # open raster
        da = xr.open_rasterio(fp)
        
        # assign time coordinate to raster
        dt = (datetime.datetime(year, 1, 1) + datetime.timedelta(day - 1))
        da = da.assign_coords(time=dt)
        
        # add current day to list of rasters
        da_list.append(da)
    # consolidate rasters
    da = xr.concat(da_list, dim='time').squeeze()
    
    # give raster a name
    da.name = data
    # apply scale factor
    da.values = da.values * eval(da.attrs['scale_factor'])
    if not is_qc:
        da.values = da.values.astype(float)
        no_data = da.attrs['nodatavals'][0]
        da.values = (np.ma.masked_equal(da.values, no_data))
    
    da = da.sortby(da.time)
    return da


def interp_da(to_interp, interp_like):
    """
    Function intended to perform temporal interpolation on the EVI data.
    
    :param to_interp: the DataArray to temporally interpolate
    :param interp_like: the DataArray with the desired temporal resolution
    
    :returns: interpolated input DA
    """
    return to_interp.interp({'time':interp_like.time}, method='linear')

In [5]:
QC_MAP = {
    'TA': [0, 4],
    'EVI': [0],
    'LST_Day': [0],
    'LST_Night': [0]
}

def filter_qa(da_name, qc_da_name):
    """
    Filters pixels from dataset based on QA info found in MODXXA2 User Guide.
    
    :param da_name: the name of the DataArray found in keys of FILE_DICT
    :param qc_da_name: the name of the QC DataArray found in the keys of FILE_DICT
    
    :returns: filtered dataset
    """
    da = open_xarray(da_name)
    qc_da = open_xarray(qc_da_name, is_qc=True)
    
    qc_filter = QC_MAP[da_name]
    
    for f in qc_filter:
        if da_name == 'TA':
            da.values[qc_da.values == f] = np.nan
        else:
            da.values[qc_da.values != f] = np.nan
    
    return da

In [6]:
def load_datasets():
    """
    Loads all datasets, does qa filtering, and interpolates where necessary.
    Saves datasets to global variables defined at top of notebook.
    """
    global TA_da
    global EVI_da
    global LST_Day_da
    global LST_Night_da
    
    TA_da = filter_qa('TA', 'TA_QC')
    print('TA loaded')
    LST_Day_da = filter_qa('LST_Day', 'LST_Day_QC')
    print('LST Day loaded')
    LST_Night_da = filter_qa('LST_Night', 'LST_Night_QC')
    print('LST Night loaded')
    EVI_da = filter_qa('EVI', 'EVI_pixel')
    EVI_da = interp_da(EVI_da, TA_da)
    print('EVI loaded')

In [7]:
load_datasets()

TA loaded
LST Day loaded
LST Night loaded
EVI loaded


In [8]:
def pixel_good(time, pixel):
    """
    Determines if a pixel is acceptable to add to a non-fire pixel patch.
    (ie not NaN in any raster and not a fire pixel (7,8,9)).
    
    :param time: the time index of the pixel
    :param pixel: the spatial index to check
    :returns: True or False (pixel is acceptable)
    """
    # check if pixel is a fire pixel
    if TA_da.values[time][pixel] in [7, 8, 9]:
        return False
    
    # check if pixel is NaN
    for da in [TA_da, EVI_da, LST_Day_da, LST_Night_da]:
        if np.isnan(da.values[time][pixel]):
            return False
    
    # pixel passed tests
    return True

def find_patch(time, curr_pix, patch, found, frontier, patch_size):
    """
    Finds a non-fire patch of pixels as an instance of no fire.
    
    :param time: raster date index to extract
    :param curr_pix: current pixel being processed
    :param patch: the patch of pixles being generated, starts as empty list
    :param found: list of visited index tuples
    :param frontier: list of pixels to invesigate, pass empty to start
    :param patch_size: number of pixels to be found for this patch
    
    :returns: list of tuples of indices in patch
    """
    # add pixel to patch
    patch.append(curr_pix)
        
    #stop when patch is appropriately sized
    if len(patch) >= patch_size:
        return patch
    
    #explore pixel neighbors
    for i in range(-1,2,1):
        for j in range(-1,2,1):
            neighbor = (curr_pix[0]+i, curr_pix[1]+j)
            # only examine pixel if not already checked
            if (neighbor not in found) and pixel_good(time, neighbor):
                frontier.append(neighbor)
                found.append(neighbor)
                
    # if no more valid pixels to check, exit
    if len(frontier) == 0:
        return patch
    
    # examine next pixel in queue
    next_pix = frontier.pop(0)
    
    # recurse
    return find_patch(time, next_pix, patch, found, frontier, patch_size)

In [9]:
def find_non_fire_patches(load_datasets=False):
    """
    Find set of non-fire patches for each day in time-series.
    
    :param load_datasets: pass True if datasets have not already been loaded
    
    :returns: dictionary of patch indices (date -> list of patches)
    """
    # set random seed
    np.random.seed(0)
    
    # load datasets
    if load_datasets:
        load_datasets()
    
    # initialize storage
    patches = {}
    
    # save dimensions of a raster
    dim1 = TA_da.shape[1]
    dim2 = TA_da.shape[2]
    
    for time in np.arange(TA_da.shape[0]):
        curr_patches = []
        found = []
        
        # find this many distinct patches for this date 
#         for _ in range(np.random.choice(np.arange(2,6), p = [0.35, 0.35, 0.15, 0.15])):
        for _ in range(5):
                
            # choose random patch size
            patch_size = 0
            while (patch_size < MIN_PATCH_SIZE) or (patch_size > MAX_PATCH_SIZE):
                # to see sample distribution generate 1000 samples and plot
                patch_size = int(np.random.lognormal(2, 1.75))
            
            # generate random start point that hasn't yet been chosen
            i, j = 0, 0
            while (not pixel_good(time, (i,j))) or ((i,j) in found):
                i = np.random.choice(np.arange(dim1))
                j = np.random.choice(np.arange(dim2))
            start = (i,j)

            # find a patch
            found.append(start)
            patch = find_patch(time, start, [], found, [], patch_size)
            curr_patches.append(patch)
            found.extend(patch)
            
        # save patches to dictionary
        patches[TA_da.time.values[time]] = curr_patches
        
    return patches

In [10]:
def compute_coordinates(y, x):
    """
    Takes a pixel coordinate and transforms to lat, lon.
    
    :param x: the horizontal coordinate of the pixel
    :param y: the vertical coordinate of the pixel
    
    :returns: lat, lon tuple"""
    ds = gdal.Open(FILE_DICT['TA'][0], gdal.GA_ReadOnly)
    gt = ds.GetGeoTransform()
    
    # extract geotransform values
    x_offset, px_w, x_rot, y_offset, y_rot, px_h = gt
    
    # compute affine position
    posX = px_w * x + x_rot * y + x_offset
    posY = y_rot * x + px_h * y + y_offset

    # shift to the center of the pixel
    posX += px_w / 2.0
    posY += px_h / 2.0
    
    # get CRS from dataset 
    crs = osr.SpatialReference()
    crs.ImportFromWkt(ds.GetProjectionRef())
    # create lat/long crs with WGS84 datum
    crsGeo = osr.SpatialReference()
    crsGeo.ImportFromEPSG(4326) # 4326 is the EPSG id of lat/long crs 
    t = osr.CoordinateTransformation(crs, crsGeo)
    (lon, lat, z) = t.TransformPoint(posX, posY)
    
    return lat, lon

In [11]:
def extract_data_from_patches(patches):
    # extract dimensions of a pixel
    pixel_length = TA_da.attrs['transform'][0]  # length of a pixel in meters
    rows = []
    
    # iterate through patches dict
    for k, v in patches.items():
        time = k
        # iterate through patches 
        for patch in v:
            # extract relevant values from each patch
            curr_row = {}
            ix = ([p[0] for p in patch], [p[1] for p in patch])
            pixel_coords = np.mean(ix, axis=1)
            lat, lon = compute_coordinates(*pixel_coords)
            curr_row['time'] = time
            curr_row['lat'] = lat
            curr_row['lon'] = lon
            curr_row['area'] = pixel_length**2 * len(patch)
            curr_row['lst_day'] = np.nanmean(LST_Day_da.sel(time=time).values[ix])
            curr_row['lst_night'] = np.nanmean(LST_Night_da.sel(time=time).values[ix])
            curr_row['evi'] = np.nanmean(EVI_da.sel(time=time).values[ix])
            curr_row['class'] = 'no_fire'
            rows.append(curr_row)
        
    return pd.DataFrame(rows)

In [12]:
## find the patches (code commented because dataset already generated)
# patches = find_non_fire_patches()

In [13]:
## turn extracted data into csv and save
# df = extract_data_from_patches(patches)
# df.to_csv('non_fire.csv', index=False)

In [14]:
df = pd.read_csv('non_fire.csv')

In [15]:
df.head()

,time,lat,lon,area,lst_day,lst_night,evi,class
0,2018-06-26,33.408854,-117.076050,1.373816e+07,307.487500,289.228750,0.216975,no_fire
1,2018-06-26,36.141667,-118.236654,1.202089e+07,305.820000,283.662857,0.242957,no_fire
2,2018-06-26,41.055741,-121.923551,5.452330e+08,300.717102,291.531937,0.398504,no_fire
3,2018-06-26,35.721354,-118.660329,4.121447e+07,310.887083,290.742083,0.265087,no_fire
4,2018-06-26,37.484083,-120.411426,8.586347e+07,317.291400,294.068200,0.202601,no_fire


In [16]:
dp_frame = pd.read_csv('fire_patches.csv').drop(['Unnamed: 0'], axis=1)

In [17]:
fire_patches = {}
for i in range(int(235/5)):
    curr_patches = []
    for j in range(5):
        curr_patches.append(eval(dp_frame.loc[i*5 + j, 'Fire Coordinates']))
    fire_patches[TA_da.time.values[i]] = curr_patches

In [18]:
fire_df = extract_data_from_patches(fire_patches)

/Users/aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice
/Users/aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:22: RuntimeWarning: Mean of empty slice
/Users/aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: Mean of empty slice


In [19]:
fire_df['class'] = 'fire'

In [20]:
fire_df['time'] = fire_df.time.dt.date

In [21]:
master_df = pd.concat([df, fire_df], axis=0, ignore_index=True)

/Users/aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [22]:
master_df.to_csv('wildfires.csv', index=False)

In [23]:
master_df['time'] = pd.to_datetime(master_df.time)

In [24]:
day = master_df.time.dt.dayofyear

In [25]:
master_df

,area,class,evi,lat,lon,lst_day,lst_night,time
0,1.373816e+07,no_fire,0.216975,33.408854,-117.076050,307.487500,289.228750,2018-06-26
1,1.202089e+07,no_fire,0.242957,36.141667,-118.236654,305.820000,283.662857,2018-06-26
2,5.452330e+08,no_fire,0.398504,41.055741,-121.923551,300.717102,291.531937,2018-06-26
3,4.121447e+07,no_fire,0.265087,35.721354,-118.660329,310.887083,290.742083,2018-06-26
4,8.586347e+07,no_fire,0.202601,37.484083,-120.411426,317.291400,294.068200,2018-06-26
5,4.293173e+06,no_fire,0.226530,36.067500,-121.286973,314.836000,290.684000,2018-07-04
6,5.151808e+06,no_fire,0.362250,35.525000,-118.555102,307.930000,291.180000,2018-07-04
7,6.010443e+07,no_fire,0.248681,38.909167,-122.604980,310.032000,290.414857,2018-07-04
8,3.434539e+06,no_fire,0.387750,39.885417,-121.552694,302.890000,288.890000,2018-07-04
9,2.575904e+06,no_fire,0.316217,41.201389,-122.694587,298.920000,287.413333,2018-07-04


In [26]:
master_df = master_df.sort_values(by='time')

In [27]:
fire_df.shape

(235, 8)

In [28]:
df.shape

(235, 8)

In [29]:
master_df.iloc[235:237,:]

,area,class,evi,lat,lon,lst_day,lst_night,time
117,3.434539e+06,no_fire,0.269367,40.30000,-121.787322,279.945,270.640,2018-12-27
116,1.717269e+07,no_fire,0.176014,36.55375,-120.032176,284.329,272.353,2018-12-27


In [30]:
master_df.time.dt

In [31]:
import time

In [32]:
master_df['day'] = pd.to_timedelta(master_df.time).dt.days - 17708

/Users/aniket/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Passing datetime64-dtype data to TimedeltaIndex is deprecated, will raise a TypeError in a future version
  """Entry point for launching an IPython kernel.


In [33]:
master_df

,area,class,evi,lat,lon,lst_day,lst_night,time,day
0,1.373816e+07,no_fire,0.216975,33.408854,-117.076050,307.487500,289.228750,2018-06-26,0
1,1.202089e+07,no_fire,0.242957,36.141667,-118.236654,305.820000,283.662857,2018-06-26,0
2,5.452330e+08,no_fire,0.398504,41.055741,-121.923551,300.717102,291.531937,2018-06-26,0
3,4.121447e+07,no_fire,0.265087,35.721354,-118.660329,310.887083,290.742083,2018-06-26,0
4,8.586347e+07,no_fire,0.202601,37.484083,-120.411426,317.291400,294.068200,2018-06-26,0
239,2.747631e+07,fire,0.295525,41.427604,-121.909096,302.388125,286.555625,2018-06-26,0
238,2.661768e+07,fire,0.294650,37.791263,-119.308978,304.324516,285.251613,2018-06-26,0
237,4.293173e+07,fire,0.355929,39.002667,-122.005704,309.440800,293.573469,2018-06-26,0
236,3.348675e+07,fire,0.297830,36.665491,-121.317327,307.558462,288.808205,2018-06-26,0
235,3.005221e+07,fire,0.316146,36.406548,-119.084158,308.053714,293.182857,2018-06-26,0


In [34]:
master_df.to_csv('wildfires_less_null.csv', index=False)

In [35]:
fire_df.isnull().sum()

area          0
class         0
evi          20
lat           0
lon           0
lst_day      36
lst_night    36
time          0
dtype: int64